# **Preprocessing**

### **Analizing Data**

Pada tahapan ini kita akan menyiapkan library yang dibutuhkan dan melakukan import class dan juga kita akan menganalisa dari karakter datanya

In [2]:
# importing library
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from wordcloud.wordcloud import WordCloud
import re
import string
import numpy as np
import pandas as pd

In [3]:
# import dataset
df = pd.read_csv("chatgpt_reviews.csv")

# membaca data
df.head()

,date,title,review,rating
0,2023-05-21 16:42:24,Much more accessible for blind users than the ...,Up to this point I’ve mostly been using ChatGP...,4
1,2023-07-11 12:24:19,"Much anticipated, wasn’t let down.",I’ve been a user since it’s initial roll out a...,4
2,2023-05-19 10:16:22,"Almost 5 stars, but… no search function",This app would almost be perfect if it wasn’t ...,4
3,2023-05-27 21:57:27,"4.5 stars, here’s why","I recently downloaded the app and overall, it'...",4
4,2023-06-09 07:49:36,"Good, but Siri support would take it to the ne...",I appreciate the devs implementing Siri suppor...,4


lalu kita akan melihat dalam setiap class ada berapa jumlah dataset nya

In [5]:
# melihat banyak data setiap class
df.rating.value_counts()

rating
5    1134
1     495
4     304
3     220
2     139
Name: count, dtype: int64

dari di atas kita dapat melihat bahwa ada pada datset kita, class yang mempunyai banyak data yang paling banyak adalah data dengan rating 5 dan rating dengan 1. dengan hal ini yang sangat mencerminkan negatif itu 1 dan yang sangat mencerminkan positif adalah 5. Karena 2-4 masih sangat kualitatif sekali.


### **Feature Selection**

Dalam tahapan ini kita akan melakukan seleksi lagi untuk fiturnya. Jadi untuk fitur yang tidak dibutuhkan, akan dihilangkan. Dalam kasus text mining kita hanya perlu document dan labelnya. fitur reviews akan menjadi document dan rating yang akan  menjadi label.

In [7]:
# untuk menghilangkan fitur
df.drop(columns=["date", "title"], inplace=True)

df.head()

,review,rating
0,Up to this point I’ve mostly been using ChatGP...,4
1,I’ve been a user since it’s initial roll out a...,4
2,This app would almost be perfect if it wasn’t ...,4
3,"I recently downloaded the app and overall, it'...",4
4,I appreciate the devs implementing Siri suppor...,4


### **Duplicated Data**

Dalam tahapan ini kita akan melihat apakah ada data yang terduplikasi atau tidak. Tujuannya adalah untuk menghilangkan data yang dapat mengganggu dalam proses modeling nya. Model akan lebih cenderung belajar yang sama dan menjadi menghafal data yang diulang ulang terus sehingga akan sulit menebak data baru. Inilah overfitting.

In [8]:
# untuk melihat data yang terduplikasi
df[df.duplicated()]

,review,rating
1351,Thanks,5
1508,Please,5
1783,Love it,5
1800,First,5
1801,First,5
1802,First,5
1827,Why?,1
1853,Very good,5
1866,Pretty good,5
1869,Great,5


dari tampilan di atas, kita dapat melihat bahwa ada data yang terduplikasi. Maka kita perlu untuk menghapusnya

In [10]:
# untuk menghapus data duplikat
df.drop_duplicates(inplace=True)

df.head()

,review,rating
0,Up to this point I’ve mostly been using ChatGP...,4
1,I’ve been a user since it’s initial roll out a...,4
2,This app would almost be perfect if it wasn’t ...,4
3,"I recently downloaded the app and overall, it'...",4
4,I appreciate the devs implementing Siri suppor...,4


### **Missing Value**

Pada tahapan ini kita akan mencari apakah ada data yang hilang atau tidak. Dalam kasus text mining jika data hilang adalah review maka kita bisa menghapus item tersebut karena jika review nya hilang tidak dapat digantikan dengan apa apa karena datanya selalu unik.

In [11]:
# untuk menghitung data kosong jenis Nan
df.isna().sum()

review    0
rating    0
dtype: int64

In [12]:
# untuk menghitung data kosong jenis null
df.isnull().sum()

review    0
rating    0
dtype: int64

In [13]:
# nilai unik rating
df.rating.unique()

array([4, 1, 3, 5, 2], dtype=int64)

melihat dari analisa di atas, kita tidak menemukan data yang kosong. Maka kita bisa mengabaikannya

### **Train Test Split**

Pada tahapan ini kita akan melakukan pemisahan antara data tes dan data trainnya. Tujuannya adalah untuk dapat menggunakan data test yang akan digunakan untuk mengetahui performa model. Dan kami memutuskan menempatkan train test split pada tahapan ini bertujuan agar bisa dilakukan pengetesan pada kasus real (benar benar bentuk text)

In [14]:
# mengembalikan x, y yang sudah di split 
def Train_Test_Split(x, y, random_seed=None, test_size=0.8):
    # banyaknya data
    n = len(x)

    # jika ada data yang pasti ke acakannya
    if random_seed:
        np.random.seed(random_seed)

    # banyaknya data test
    test_size = int(test_size*n)

    # matriks random untuk menentukan index data train dan data test
    indices = np.random.permutation(n)

    # menentukan index
    train_indices, test_indices = indices[test_size:], indices[:test_size]

    return x.iloc[train_indices], x.iloc[test_indices], y.iloc[train_indices], y.iloc[test_indices]

# menentukan x dan y nya
x = df["review"]
y = df["rating"]


# mengimplementasikan pada data
X_train, X_test, y_train, y_test = Train_Test_Split(x, y, random_seed=42, test_size=0.8)

X_train

93      I have had an excellent experience using ChatG...
1008    I thought I was mad, and kept resetting my pas...
1070    For a company that’s uses data to build and ru...
1092    Simple, easy and free no subscription required...
2111                                    必须要gmail才可以注册吗 裂开
                              ...                        
1640                               I like app it help lot
1095    After numerous cross referenced questions over...
1130    Would love it if yall allowed us (Ethiopians) ...
1294          I hope to adapt to the iPad and add Chinese
860     Great app. I just wish there was an option to ...
Name: review, Length: 453, dtype: object

In [31]:
test = ["halo", "kenapa", "itu", "itu", "kenapa", "my"]
corpusa = {"halo": "hello", "itu": "that", "kenapa":"why"}
# [ini.replace(ini, corpusa[ini]) if ini in corpusa else ini for ini in test  ]
[corpusa[ini] if ini in corpusa else ini for ini in test]

['hello', 'why', 'that', 'that', 'why', 'my']

### **Case Folding**

Pada tahapan ini

### **Tokenize**

### **Normalize**

In [ ]:
# kata yang perlu dilakukan normalize
normalize_word = {}

# akan mengembalikan document yang sudah dinormalize
def normalize(doc):
    return [normalize_word[word] if word in normalize_word else word for word in doc]


### **Stopwords**

### **Stemming**

# **Pembobotan Kata**

### **Analize Frequency Word**

### **Document Frequency**

### **Term Frequency**

### **Inverse Document Frequency**

### **TF - IDF**

# **Modeling**

### **Create Class**

### **Training**

### **Testing**

### **Conclusion**